# Lab 6: Transformers and LLMs
```
- [S25] Advanced Machine Learning, Innopolis University
- Teaching Assistant: Gcinizwe Dlamini
```
<hr>


```
Lab Plan
1. Transformers (translation architecture)
2. Self-Attention
3. Multi-headed attention
4. Positional Encoding
5. Transfomer Encoder Part
6. Application of Transformers
7. Self practice tasks
```

<hr>


# 1. Transformers

* [Attention Is All You Need](https://arxiv.org/pdf/1706.03762.pdf) -- Original paper on attention

![](http://jalammar.github.io/images/t/The_transformer_encoder_decoder_stack.png)

## 1.1 Transfomer Encoder

The encoder contains a self-attention layer – a layer that helps the encoder look at other words in the input sentence as it encodes a specific word. <br>
PyTorch implementation : `nn.TransformerEncoder` and `nn.TransformerEncoderLayer` <br>
**The main goal is to efficiently encode the data**

  1         |  2
:-------------------------:|:-------------------------:
![](http://jalammar.github.io/images/t/encoder_with_tensors.png)  |  ![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)


## 2. Self-Attention

**Keep in mind : The main goal is to encode the data in a much more efficient way** In other words is to create meaningful embeddings<br>
- As the model processes each word (each position in the input sequence), self attention allows it to look at other positions in the input sequence for clues that can help lead to a better encoding for this word.


**How does Self-Attention work?**

Steps:
1. For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.
  - What are the **`Query`** vector, a **`Key`** vector, and a **`Value`** vector? : They're abstractions that are useful for calculating attention... They are a breakdown of the word embeddings
2. Calculating self-attention score from **`Query`** **`Key`** vector.
3. Divide the scores by 8 (This leads to having more stable gradients)
4. Pass the result through a softmax operation (softmax score determines how much each word will be expressed at this position)
5. Multiply each value vector by the softmax score
6. Sum up the weighted value vectors

### Step 1

For each word, we create a **`Query`** vector, a **`Key`** vector, and a **`Value`** vector.

![](http://jalammar.github.io/images/t/transformer_self_attention_vectors.png)

In [1]:
import torch

# simple sequence = I will pass AML midterm
simple_sequence_embedding = torch.rand((5, 512))

# Create weight matrices
Wq = torch.normal(0,0.5, (512, 7))
Wk = torch.normal(0,0.1, (512, 7))
Wv = torch.normal(0,0.2, (512, 7))


# Create key, query and value for each word in the senetence
queries = simple_sequence_embedding @ Wq # (5, 512) @ (512, 7) = (5, 7)
keys = simple_sequence_embedding @ Wk # (5, 512) @ (512, 7) = (5, 7)
values = simple_sequence_embedding @ Wv# (5, 512) @ (512, 7) = (5, 7)

### Step 2

Calculating self-attention score from **`Query`** and **`Key`** vector

In [2]:
scores = torch.matmul(queries, keys.T) # (5, 7) @ (5, 7).T = (5, 5)
scores

tensor([[-10.2264, -38.4249,  -4.4373, -36.7984, -36.3380],
        [-16.3617, -25.3481,  -6.0481, -38.8037, -35.3986],
        [-24.5571, -40.8960, -12.1657, -64.3527, -42.7539],
        [-13.4427, -34.4612, -12.9985, -48.1787, -38.3386],
        [-11.5817, -35.6553,  -2.3780, -41.0603, -46.9878]])

### Step 3
Divide the scores by 8 (This leads to having more stable gradients)

In [4]:
# Divide the result to stabilize the gradients
scores = scores / 8 # Divide the result by 8 to stabilize the gradients
scores

tensor([[-0.4832, -1.8154, -0.2096, -1.7386, -1.7168],
        [-0.7730, -1.1976, -0.2857, -1.8333, -1.6724],
        [-1.1602, -1.9322, -0.5748, -3.0404, -2.0199],
        [-0.6351, -1.6281, -0.6141, -2.2762, -1.8113],
        [-0.5472, -1.6846, -0.1123, -1.9399, -2.2200]])

### Step 4

Pass the result through a softmax operation

In [5]:
scores =  torch.softmax(scores, dim=1)
scores

tensor([[0.3170, 0.0836, 0.4167, 0.0903, 0.0923],
        [0.2478, 0.1621, 0.4034, 0.0858, 0.1008],
        [0.2608, 0.1205, 0.4684, 0.0398, 0.1104],
        [0.3456, 0.1280, 0.3529, 0.0670, 0.1066],
        [0.3029, 0.0971, 0.4679, 0.0752, 0.0569]])

### Step 5 & 6

* Multiply each value vector by the softmax score
* Sum up the weighted value vectors



In [6]:
# Multiply each value vector by the corresponding softmax score
z = torch.matmul(scores, values)  # (5, 5) @ (5, 7) -> (5, 7)

## 3. Multi-headed attention

**GOAL**:
1. Expand the model's ability to focus on different positions
2. Provide the attention layer multiple “representation subspaces”

**Attention with $N$ just means repeating self attention algorithm $N$ times and joining the results**


![](https://data-science-blog.com/wp-content/uploads/2022/01/mha_img_original.png)

**Multi-headed attention steps:**
1. Same as self-attention calculation, just n different times with different weight matrices
2. Condense the $N$ z metrices down into a single matrix by concatinating the matrices then multiply them by an additional weights matrix `WO`

Now the output z metrix is fed to the FFNN

In [8]:
from torch import Tensor
import torch.nn.functional as f
from torch import nn


def scaled_dot_product_attention(query, key, value):
  temp = query.bmm(key.transpose(1, 2))
  scale = query.size(-1) ** 0.5
  softmax = f.softmax(temp / scale, dim=-1)
  return softmax.bmm(value)

### 3.1 Attention head

In [9]:
class AttentionHead(nn.Module):
    def __init__(self, dim_in, dim_q, dim_k):
        super().__init__()
        self.q = nn.Linear(dim_in, dim_q)  # Linear transformation for Query
        self.k = nn.Linear(dim_in, dim_k)  # Linear transformation Key
        self.v = nn.Linear(dim_in, dim_k)  # Linear transformation Value

    def forward(self, query, key, value):
        q = self.q(query)  
        k = self.k(key)    
        v = self.v(value)  

        return scaled_dot_product_attention(q, k, v)

### 3.2 Multi Head Attention

**Task:** Implement multi-head attention model

In [10]:
class MultiHeadAttention(nn.Module):
  def __init__(self, number_of_heads, dim_in, dim_q, dim_k):
    super().__init__()
    self.heads = nn.ModuleList([AttentionHead(dim_in, dim_q, dim_k) for _ in range(number_of_heads)])
    self.linear = nn.Linear(number_of_heads* dim_k, dim_in)

  def forward(self, query, key, value):
    z = self.linear(torch.cat([h(query, key, value) for h in self.heads], dim=-1))
    return z


## 4. Positional Encoding

A way to account for the order of the words in the input sequence. A transformer adds a vector to each input embedding which helps it determine the position of each word. <br>
**Goal** : preserving information about the order of tokens  

Positional Encoding they can either be learned or fixed a priori.

Proposed approach from original paper : describe a simple scheme for fixed positional encodings based on sine and cosine functions

In [11]:
def position_encoding(seq_len, dim_model, device):
  pos = torch.arange(seq_len, dtype=torch.float, device=device).reshape(1, -1, 1)
  dim = torch.arange(dim_model, dtype=torch.float, device=device).reshape(1, 1, -1)
  phase = pos / (1e4 ** (dim / dim_model))

  return torch.where(dim.long() % 2 == 0, torch.sin(phase), torch.cos(phase))

![](http://jalammar.github.io/images/t/transformer_resideual_layer_norm_2.png)

## 5. Transfomer Encoder Part
### 5.1Encoder Feed Forward

In [12]:
def feed_forward(dim_input = 512, dim_feedforward = 2048):
  return nn.Sequential(nn.Linear(dim_input, dim_feedforward), nn.ReLU(), nn.Linear(dim_feedforward, dim_input))

### 5.2 Encoder Residual

From the original paper the author implementation

In [13]:
class Residual(nn.Module):
  def __init__(self, sublayer, dimension, dropout = 0.1):
    super().__init__()
    self.sublayer = sublayer
    self.norm = nn.LayerNorm(dimension)
    self.dropout = nn.Dropout(dropout)

  def forward(self, *tensors):
    return self.norm(tensors[0] + self.dropout(self.sublayer(*tensors)))

### 5.3 Putting the Encoder layer together

In [14]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, dim_model = 512, num_heads = 6, dim_feedforward = 2048, dropout = 0.1):
    super().__init__()
    dim_q = dim_k = max(dim_model // num_heads, 1)
    self.attention = Residual(MultiHeadAttention(num_heads, dim_model, dim_q, dim_k),
                              dimension=dim_model, dropout=dropout)
    self.feed_forward = Residual(feed_forward(dim_model, dim_feedforward), dimension=dim_model, dropout=dropout)

  def forward(self, src):
    src = self.attention(src, src, src)
    return self.feed_forward(src)

## 5.4 Putting together transfomer Encoder part

In [15]:
class TransformerEncoder(nn.Module):
  def __init__(self, num_layers = 12, dim_model = 512, num_heads = 4, dim_feedforward = 2048, dropout: float = 0.1):
    super().__init__()
    self.layers = nn.ModuleList([TransformerEncoderLayer(dim_model, num_heads, dim_feedforward, dropout) for _ in range(num_layers) ])

  def forward(self, src):
    seq_len, dimension = src.size(1), src.size(2)
    src += position_encoding(seq_len, dimension)
    for layer in self.layers:
      src = layer(src)

    return src

# The Decoder Side

The encoder start by processing the input sequence. The output of the top encoder is then transformed into a set of attention vectors K and V. These are to be used by each decoder.

The “Encoder-Decoder Attention” layer works just like multiheaded self-attention, except it creates its Queries matrix from the layer below it, and takes the `Keys` and `Values` matrix from the output of the encoder stack.

![](https://jalammar.github.io/images/t/transformer_resideual_layer_norm_3.png)


## Decoder layer



**Task**: implement the decoder layer

In [ ]:
class DecoderLayer(nn.Module):
    def __init__(self, dim_model=512, num_heads=8, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        dim_q = dim_k = max(dim_model // num_heads, 1)

        # Masked Multi-Head Attention (for Decoder)
        self.masked_attention = Residual(
            MultiHeadAttention(num_heads, dim_model, dim_q, dim_k),
            dimension=dim_model, dropout=dropout
        )

        # Encoder-Decoder Attention
        self.encoder_decoder_attention = Residual(
            MultiHeadAttention(num_heads, dim_model, dim_q, dim_k),
            dimension=dim_model, dropout=dropout
        )

        # Feed Forward Network
        self.feed_forward = Residual(
            feed_forward(dim_model, dim_feedforward),
            dimension=dim_model, dropout=dropout
        )

    def forward(self, tgt, memory):
        # Masked attention for the decoder
        tgt = self.masked_attention(tgt, tgt, tgt)
        
        # Encoder-Decoder Attention: tgt - это Queries, memory - это Keys и Values
        tgt = self.encoder_decoder_attention(tgt, memory, memory)
        
        return self.feed_forward(tgt)


## 6. Application of Transfomers

We will look at sentiment analysis

In [17]:
%pip install transformers


   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   --- ------------------------------------ 0.8/10.0 MB 6.7 MB/s eta 0:00:02
   ------------- -------------------------- 3.4/10.0 MB 10.6 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/10.0 MB 14.2 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 14.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 22.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
import transformers

class Transformer(nn.Module):

  def __init__(self, output_dim):
    super().__init__()
    self.transformer = transformers.AutoModel.from_pretrained('bert-base-uncased')
    for param in self.transformer.parameters():
        param.requires_grad = False

    hidden_dim = self.transformer.config.hidden_size
    self.fc = nn.Linear(hidden_dim, output_dim)


  def forward(self, text):
    # text = [batch size, seq len]
    output = self.transformer(text, output_attentions=True)
    hidden = output.last_hidden_state
    # hidden = [batch size, seq len, hidden dim]
    attention = output.attentions[-1]
    # attention = [batch size, n heads, seq len, seq len]
    cls_hidden = hidden[:, 0, :]
    prediction = self.fc(torch.tanh(cls_hidden))

    return prediction

## 7. Tasks

```
Task 1
- Using the above implementation code the decoder layer and assemble a full transformer model
```

<hr>

```
Task 2
- Implement, train and test a Transfomer model (use pytorch layers)for Part-of-speech tagging task.
```

**Task 2 Datasets**: [Train](https://www.dropbox.com/s/x9n6f9o9jl7pno8/train_pos.txt?dl=1), [Test](https://www.dropbox.com/s/v8nccvq7jewcl8s/test_pos.txt?dl=1)


In [19]:
# Step 10: Full Transformer Model
class Transformer(nn.Module):
    def __init__(self, input_dim, output_dim, num_heads=8, num_layers=6, dim_model=512, dim_feedforward=2048, dropout=0.1):
        super().__init__()
        self.encoder = TransformerEncoder(num_layers, dim_model, num_heads, dim_feedforward, dropout)
        self.decoder = DecoderLayer(dim_model, num_heads, dim_feedforward, dropout)
        self.fc_out = nn.Linear(dim_model, output_dim)

    def forward(self, src, tgt):
        # src = input sequence, tgt = target sequence (for Decoder)
        memory = self.encoder(src)  # Encoder output (memory)
        output = self.decoder(tgt, memory)  # Decoder output
        return self.fc_out(output)  # Output layer